In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file only if it exists
dotenv_path = '/workspace/general_projects/database_generator/.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    print(f"Loaded environment variables from {dotenv_path}")
else:
    print(f"No .env file found at {dotenv_path}, relying on system environment variables.")

# Access the environment variable, with a fallback
path_to_logs = os.getenv('PATH_TO_SAVE_THE_LOGS')
print(f"Logs will be saved to: {path_to_logs}")

# Access the environment variable, with a fallback
path_to_logs = os.getenv('PATH_TO_SAVE_THE_OUTCOMES')
print(f"Logs will be saved to: {path_to_logs}")

Loaded environment variables from /workspace/general_projects/database_generator/.env
Logs will be saved to: /workspace/general_projects/database_generator/notebooks/tmp


In [4]:
path_to_logs

'/workspace/general_projects/database_generator/notebooks/tmp'

In [6]:
import os
print(os.getenv("PATH_TO_SAVE_THE_LOGS"))

/workspace/general_projects/database_generator/notebooks/tmp


In [7]:
print(os.environ)

environ({'HOSTNAME': 'c38c0673b4b7', 'HOME': '/root', 'PYTHONUNBUFFERED': '1', 'GPG_KEY': 'A035C8C19219BA821ECEA86B64E628F8D684696D', 'PYTHON_SHA256': '07a4356e912900e61a15cb0949a06c4a05012e213ecd6b4e84d0f67aabbee372', 'PATH': '/workspace/general_projects/database_generator/.venv/bin:/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807/bin/remote-cli:/root/.pyenv/shims:/root/.pyenv/bin:/root/.local/bin:/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 'C.UTF-8', 'SHELL': '/bin/bash', 'PYTHON_VERSION': '3.11.10', 'PWD': '/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807', 'PYENV_ROOT': '/root/.pyenv', 'VSCODE_CWD': '/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807', 'VSCODE_NLS_CONFIG': '{"userLocale":"en","osLocale":"en","resolvedLanguage":"en","defaultMessagesFile":"/vscode/vscode-server/bin/linux-x64/e8653663e8840adaf45af01eab5c627a5af81807/out/nls.messages.json","locale

In [9]:
from database_generator.data_generator import IndustrialPumpData

start_datetime = datetime(2023, 1, 1, 0, 0, tzinfo=timezone.utc)
end_datetime = datetime(2023, 1, 2, 0, 0, tzinfo=timezone.utc)
frequency = '1h'
seed_for_random = 42
pump_data = IndustrialPumpData(
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    frequency=frequency,
    seed_for_random=seed_for_random
)

OSError: Environment variable 'PATH_TO_SAVE_THE_OUTCOMES' is not defined. Please set it in your environment or .env file.